In [62]:
import pickle as pkl
import numpy as np
import random
def load_pkl(file_name):
    with open(file_name, 'rb') as f:
        data = pkl.load(f)
    return data

In [63]:
sonar_data = load_pkl("sonar_data.pkl")

In [64]:
mines = sonar_data['m']
dataset = []
for mine in mines:
    dataset.append([mine, 1])

rocks = sonar_data['r']
for rock in rocks:
    dataset.append([rock, -1])

random.shuffle(dataset)
len(dataset[0][0])

60

In [65]:
import numpy as np
import random

def save_pkl(file_name, data):
            with open(file_name, 'wb') as f:
                pkl.dump(data, f)

class Neuron:

    def __init__(self, dimension=1, weights=None, bias=None, activation=(lambda x: x), predict=(lambda x: x)):
        self._dim = dimension
        self.w = weights or np.array([random.random()*(-1)**random.randint(0, 1) for _ in range(self._dim)])
        self.w = np.array([float(w) for w in self.w])
        self.b = bias if bias is not None else random.random()*(-1)**random.randint(0, 1)
        self.b = float(self.b)
        self._a = activation
        self.predict = predict.__get__(self)

    def __str__(self):
        return "Simple cell neuron\n\
        \tInput dimension: %d\n\
        \tBias: %f\n\
        \tWeights: %s\n\
        \tActivation: %s" % (self._dim, self.b, self.w, self._a.__name__)

    def __call__(self, x):
    
        yhat = self._a(np.dot(self.w, np.array(x)) + self.b)
        return yhat
    
class Trainer:

    def __init__(self, dataset, model, loss):
    
        self.dataset = dataset
        self.model = model
        self.loss = loss

    def validate(self, data):

        results = [self.loss(self.model.predict(x), y) for x, y in data]
        return float(sum(result for result in results))/float(len(results))
    
    def accuracy(self, data):
        return 100*float(sum([1 for x, y in data if self.model.predict(x) == y]))/float(len(data))

    def train(self, lr, ne):

        print("training model on data...")
        accuracy = self.accuracy(self.dataset)
        print("initial accuracy: %.3f" % (accuracy))
        for epoch in range(ne):
            for d in self.dataset:
                x, y = d
                x = np.array(x)
                yhat = self.model(x)
                error = y - yhat
                self.model.w += lr*(y-yhat)*x
                self.model.b += lr*(y-yhat)
            accuracy = self.accuracy(self.dataset)
            print('>epoch=%d, learning_rate=%.3f, accuracy=%.3f' % (epoch+1, lr, accuracy))
        print("training complete")
        print("final accuracy: %.3f" % (self.accuracy(self.dataset)))

# activation functions

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def perceptron(z):
    return -1 if z<=0 else 1

# loss functions

def lrloss(yhat, y):
    return -1.0*(y*np.log(yhat)+(1-y)*np.log(1-yhat))

def ploss(yhat, y):
    return max(0, -yhat*y)

# prediction functions

def ppredict(self, x):
    return self(x)

def lrpredict(self, x):
    return 1 if self(x)>0.5 else 0
    
# extra

def softmax(z):
    return np.exp(z) / np.sum(np.exp(z))

In [66]:
neuron = Neuron(dimension = len(dataset[0][0]), activation = perceptron, predict = ppredict)
trainer = Trainer(dataset, neuron, ploss)
updates = trainer.train(0.01, 8000)

training model on data...
initial accuracy: 46.635
>epoch=1, learning_rate=0.010, accuracy=49.038
>epoch=2, learning_rate=0.010, accuracy=63.942
>epoch=3, learning_rate=0.010, accuracy=61.538
>epoch=4, learning_rate=0.010, accuracy=69.712
>epoch=5, learning_rate=0.010, accuracy=70.673
>epoch=6, learning_rate=0.010, accuracy=72.115
>epoch=7, learning_rate=0.010, accuracy=73.558
>epoch=8, learning_rate=0.010, accuracy=73.558
>epoch=9, learning_rate=0.010, accuracy=71.635
>epoch=10, learning_rate=0.010, accuracy=72.115
>epoch=11, learning_rate=0.010, accuracy=75.481
>epoch=12, learning_rate=0.010, accuracy=75.481
>epoch=13, learning_rate=0.010, accuracy=77.404
>epoch=14, learning_rate=0.010, accuracy=75.962
>epoch=15, learning_rate=0.010, accuracy=77.404
>epoch=16, learning_rate=0.010, accuracy=76.923
>epoch=17, learning_rate=0.010, accuracy=76.923
>epoch=18, learning_rate=0.010, accuracy=77.404
>epoch=19, learning_rate=0.010, accuracy=74.519
>epoch=20, learning_rate=0.010, accuracy=76.44

In [67]:
trainer_1 = list([neuron.b, *neuron.w])
save_pkl('sonar_bw.pkl', trainer_1)

with open('sonar_bw.pkl', 'rb') as f:
    loadedneuron = pkl.load(f)
    
len(loadedneuron)

61

In [68]:
print(loadedneuron)

[-0.978768498557326, 3.3511545166153662, -1.4401817854123216, -3.15837383296481, 3.510885284268141, -0.17467554421393425, 0.7279040901577367, -2.2551317237721578, 0.46988140204224665, 0.6980620599025197, -0.8567161438607757, 1.1076498314120176, 0.6956614504612143, 0.18441465402360888, -0.6694051504305462, 0.9798909491287509, -1.066429878870627, -0.8806606391473315, 0.947173891077489, -0.5479527878834912, 1.3028039174048376, -1.629711820188371, 1.9678986684487436, -1.3659706026068985, 1.4383521576802103, -0.10550921841221099, -0.8948449120448982, 0.916736904168914, -0.6067457670631093, 0.062158610489854375, 1.2118689132115077, -1.8794546966285939, 0.7402139603007399, 0.7608006802307903, -1.3428163045871864, 1.1978454477722733, -1.2258937679076958, 0.12087913429223239, -0.12019397524033579, 1.0844865281879197, -1.5437453019031795, 0.464725864988818, 0.11815398964919208, -0.1388890395663341, 0.5389676260167825, 0.3152409889327042, 0.7548051863300667, -1.6134666306743164, 2.836754375158656